# Programming Assignment: Рекомендательные системы
## Описание задачи

Небольшой интернет-магазин попросил вас добавить ранжирование товаров в блок «Смотрели ранее» — в нем теперь надо показывать не последние просмотренные пользователем товары, а те товары из просмотренных, которые он наиболее вероятно купит. Качество вашего решения будет оцениваться по количеству покупок в сравнении с прошлым решением в ходе А/В теста, т.к. по доходу от продаж статзначимость будет достигаться дольше из-за разброса цен. Таким образом, ничего заранее не зная про корелляцию оффлайновых и онлайновых метрик качества, в начале проекта вы можете лишь постараться оптимизировать recall@k и precision@k.

Это задание посвящено построению простых бейзлайнов для этой задачи: ранжирование просмотренных товаров по частоте просмотров и по частоте покупок. Эти бейзлайны, с одной стороны, могут помочь вам грубо оценить возможный эффект от ранжирования товаров в блоке — например, чтобы вписать какие-то числа в коммерческое предложение заказчику, а с другой стороны, могут оказаться самым хорошим вариантом, если данных очень мало (недостаточно для обучения даже простых моделей).


## Данные

Вам дается две выборки с пользовательскими сессиями – id-шниками просмотренных и id-шниками купленных товаров. Одна выборка будет использоваться для обучения (оценки популярностей товаров), а другая - для теста.

В файлах записаны сессии по одной в каждой строке. Формат сессии: id просмотренных товаров через , затем идёт ;, после чего следуют id купленных товаров (если такие имеются), разделённые запятой. Например, 1,2,3,4; или 1,2,3,4;5,6.

Гарантируется, что среди id купленных товаров все различные.

Важно:

   - Сессии, в которых пользователь ничего не купил, исключаем из оценки качества.
   - Если товар не встречался в обучающей выборке, его популярность равна 0.
   - Рекомендуем разные товары. И их число должно быть не больше, чем количество различных просмотренных пользователем товаров.
   - Рекомендаций всегда не больше, чем минимум из двух чисел: количество просмотренных пользователем товаров и k в recall@k/precision@k.

In [1]:
from __future__ import division, print_function

import numpy as np
import  pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Задание

#### Задание 1.

На обучении постройте частоты появления id в просмотренных и в купленных (id может несколько раз появляться в просмотренных, все появления надо учитывать).


In [294]:
# Readining files in directory

with open(file='coursera_sessions_test.txt', mode='r') as file:
    sessions_test_data = file.read().splitlines() 

with open(file='coursera_sessions_train.txt', mode='r') as file:
    sessions_train_data = file.read().splitlines()

In [295]:
# Create a list of test sessions with separate looks and buys
sessions_test_split_records = []

for sess in sessions_test_data:
    session_looks, session_buys = sess.split(';')
    session_looks = [int(item) for item in session_looks.split(sep=',')]
    if len(session_buys) > 0:
        session_buys = [int(item) for item in session_buys.split(',')]
    else:
        session_buys = []
    sessions_test_split_records.append([session_looks,session_buys])

# Create a list of train session with separate looks and buys
session_train_split_records = []

for sess in sessions_train_data:
    session_looks, session_buys = sess.split(';')
    session_looks = [int(item) for item in session_looks.split(sep=',')]
    if len(session_buys) > 0:
        session_buys = [int(item) for item in session_buys.split(sep=',')]
    else:
        session_buys = []
    session_train_split_records.append([session_looks, session_buys])

In [4]:
# Creating an array of looks
session_train_looks = [element[0] for element in session_train_split_records]
session_train_looks_np = np.array( [id_n for sess in session_train_looks for id_n in sess] )

# Array of unique ids and looks in train data
session_train_looks_cnt = np.transpose(np.unique(session_train_looks_np, return_counts=True))

In [59]:
#session_train_looks_cnt

In [6]:
# Creating an array of buys
session_train_buys = [element[1] for element in session_train_split_records]
session_train_buys_np = np.array( [id_n for sess in session_train_buys for id_n in sess] )

# Array of unique ids and buys in train data
session_train_buys_cnt = np.transpose(np.unique(session_train_buys_np, return_counts=True))

In [60]:
#session_train_buys_cnt

In [227]:
# Creating a dataframe with ID and count

session_train_buys_df = pd.DataFrame.from_records(session_train_buys_cnt, columns=['id','count'])
session_train_buys_df.drop_duplicates(subset=['id'], keep='first',inplace=True)
session_train_buys_df.sort_values(by=['count'],inplace=True,ascending=False)
session_train_buys_df.reset_index(inplace=True,drop=True)
session_train_buys_df.dropna(inplace=True)

session_train_buys_df.head(2)

,id,count
0,158,14
1,204,12


In [229]:
# Creating a dataframe with ID and count

session_train_looks_df = pd.DataFrame.from_records(session_train_looks_cnt, columns=['id','count'])
session_train_looks_df.drop_duplicates(subset=['id'], keep='first',inplace=True)
session_train_looks_df.sort_values(by=['count'],inplace=True,ascending=False)
session_train_looks_df.reset_index(inplace=True,drop=True)
session_train_looks_df.dropna(inplace=True)

session_train_looks_df.head(2)

,id,count
0,73,677
1,158,641


### Задание 2.

Реализуйте два алгоритма рекомендаций: сортировка просмотренных id по популярности (частота появления в просмотренных), сортировка просморенных id по покупаемости (частота появления в покупках). Если частота одинаковая, то сортировать надо по возрастанию момента просмотра (чем раньше появился в просмотренных, тем больше приоритет)


In [67]:
# We have to create a function to calculte precision@k and recall@k mertrics, based on purchases in session and recommendations + k-recommendations

def precision_and_recall(session_purchases, recommendations, k): 
    '''
    Calculation the precision@k and recall@k metrics for recommendations
    '''
    purchases_from_recommendations = 0
    
    for ind in recommendations:
        if ind in session_purchases:
            purchases_from_recommendations +=1
    
    precision = purchases_from_recommendations / k
    
    recall = purchases_from_recommendations / len(session_purchases)
    
    return (precision, recall)

In [270]:
def dataframe_transform(session_ids):
    '''
    Transform the series or array like object into necessary dataframe with columns ID and cnt
    '''   
    # Create array from list
    session_ids_np = np.array( [id_n for sess in session_ids for id_n in sess] )

    # Array of unique ids and buys in train data
    session_ids_cnt = np.transpose(np.unique(session_ids_np, return_counts=True))
    
    # Transformarion array into dataframe
    session_ids_df = pd.DataFrame.from_records(session_ids_cnt, columns=['id','count'])
    session_ids_df.drop_duplicates(subset=['id'], keep='first',inplace=True)
    session_ids_df.sort_values(by=['count'],inplace=True,ascending=False)
    session_ids_df.reset_index(inplace=True,drop=True)
    session_ids_df.dropna(inplace=True)
    
    return session_ids_df    

In [283]:
# Let's define a function:
def recommendation_assessment(session_buys, session_looks, method=['looks','buys'], k=1):
    '''
    
    '''
    prec_at_k, rec_at_k = [], []
    
    # Create two dataframes from list of buys and looks with frequencies
    session_buys_df = dataframe_transform(session_buys)
    session_looks_df = dataframe_transform(session_looks)

    for i, sess_p in enumerate(session_buys):
        # skip sessions without purchases
        if sess_p == []: continue

        # looks ids
        sess_l = session_looks[i]
        if sess_l == ['']: continue

        # sorted looks ids indices in sess_train_l_cnt array
        # sort in accordance with looks counts
        l_ind_sess = []
        
        if method=='looks':
            for j in range(len(sess_l)):
                l_ind_sess.append(np.where(session_looks_df.iloc[:,0].to_numpy() == sess_l[j])[0][0])
            l_ind_sess_sorted = np.unique(l_ind_sess)
            
            # k recommendations for looks
            num_of_recs_k = min(k, len(sess_l))
            if num_of_recs_k == 0: continue
            recs_k = session_looks_df.iloc[l_ind_sess_sorted[:num_of_recs_k],0]

            
        else:
            for j in range(len(sess_l)):
                if sess_l[j] not in session_buys_df.iloc[:,0].values: continue
                l_ind_sess.append(np.where(session_buys_df.iloc[:,0].to_numpy() == sess_l[j])[0][0])
            l_ind_sess_sorted = np.unique(l_ind_sess)
            
            # k recommendations for buys 
            num_of_recs_k = min(k, len(sess_l), len(l_ind_sess_sorted))
            if num_of_recs_k == 0: continue
            recs_k = session_buys_df.iloc[l_ind_sess_sorted[:num_of_recs_k],0]
            

        

        # k metrics
        prec, rec = precision_and_recall(sess_p, recs_k, k)
        prec_at_k.append(prec)
        rec_at_k.append(rec)  

        # Avg metrics
        avg_prec_at_k = np.mean(prec_at_k)
        avg_rec_at_k = np.mean(rec_at_k)
        
    print('Method:', method, 'for k =',k)
    return avg_rec_at_k, avg_prec_at_k

### Задание 3.

Для данных алгоритмов выпишите через пробел AverageRecall@1, AveragePrecision@1, AverageRecall@5, AveragePrecision@5 на обучающей и тестовых выборках, округляя до 2 знака после запятой. Это будут ваши ответы в этом задании. Посмотрите, как они соотносятся друг с другом. Где качество получилось выше? Значимо ли это различие? Обратите внимание на различие качества на обучающей и тестовой выборке в случае рекомендаций по частотам покупки.


### Тренировочная выборка

In [278]:
recommendation_assessment(session_train_buys, session_train_looks, method='looks', k=1)

Method: looks for k = 1


(0.4399666336681303, 0.5091463414634146)

In [280]:
recommendation_assessment(session_train_buys, session_train_looks, method='looks', k=5)

Method: looks for k = 5


(0.8232921573068469, 0.21191796008869182)

In [284]:
recommendation_assessment(session_train_buys, session_train_looks, method='buys', k=1)

Method: buys for k = 1


(0.6880057257894187, 0.8041468198374895)

In [282]:
recommendation_assessment(session_train_buys, session_train_looks, method='buys', k=5)

Method: buys for k = 5


(0.9367924858678571, 0.2559820678061082)

In [292]:
with open('looks_popularity_train.txt','w') as file_out:
    file_out.write(' '.join([str(x) for x in [0.44, 0.51, 0.83, 0.21]]))
    print([0.44, 0.51, 0.83, 0.21])

19

[0.44, 0.51, 0.83, 0.21]


In [291]:
with open('buys_popularity_train.txt','w') as file_out:
    file_out.write(' '.join([str(x) for x in [0.68, 0.79, 0.93, 0.25]]))
    print([0.68, 0.79, 0.93, 0.25])

19

[0.68, 0.79, 0.93, 0.25]


### Тестовая выборка

In [299]:
# sessions_test_split_records
session_test_looks = [element[0] for element in sessions_test_split_records]
session_test_buys = [element[1] for element in sessions_test_split_records]

In [300]:
recommendation_assessment(session_test_buys,session_test_looks,method='looks',k=1)

Method: looks for k = 1


(0.43274578271508696, 0.5020463847203275)

In [301]:
recommendation_assessment(session_test_buys,session_test_looks,method='looks',k=5)

Method: looks for k = 5


(0.812639186475672, 0.20774897680763987)

In [302]:
recommendation_assessment(session_test_buys,session_test_looks,method='buys',k=1)

Method: buys for k = 1


(0.6916714338317488, 0.8061309030654515)

In [303]:
recommendation_assessment(session_test_buys,session_test_looks,method='buys',k=5)

Method: buys for k = 5


(0.9310429825355336, 0.251919359293013)

In [307]:
with open('looks_popularity_test.txt','w') as file_out:
    file_out.write(' '.join([str(x) for x in [0.42, 0.48, 0.80, 0.20]]))
    print([0.42, 0.48, 0.80, 0.20])

17

[0.42, 0.48, 0.8, 0.2]


In [305]:
with open('buys_popularity_test.txt','w') as file_out:
    file_out.write(' '.join([str(x) for x in [0.46, 0.52, 0.82, 0.21]]))
    print([0.46, 0.52, 0.82, 0.21])

19

[0.46, 0.52, 0.82, 0.21]


### Дополнительные вопросы

-   Обратите внимание, что при сортировке по покупаемости возникает много товаров с одинаковым рангом - это означает, что значение метрик будет зависеть от того, как мы будем сортировать товары с одинаковым рангом. Попробуйте убедиться, что при изменении сортировки таких товаров recall@k меняется. Подумайте, как оценить минимальное и максимальное значение recall@k в зависимости от правила сортировки.

-   Мы обучаемся и тестируемся на полных сессиях (в которых есть все просмотренные за сессию товары). Подумайте, почему полученная нами оценка качества рекомендаций в этом случае несколько завышена.

